In [ ]:
#Standard inmorts 
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from scipy import io
from scipy import stats
from scipy.spatial import distance_matrix


#the rest should be ML 

from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# K-Means 
from sklearn.cluster import KMeans


# Pipeline stuff
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

#K-Prototypes
import sklearn.metrics as metrics
from sklearn.metrics import pairwise_distances

from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler

## settings

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

mypal2 =['#FEAEFE', '#FC05FB', '#B4FFE4', '#3FFEBA', '#1F363D', '#2B4570', '#3A2D32', '#706F6F']

# 1. Read Data

In [ ]:
#df1 = pd.read_feather('../Data/D3_EBAN_raw.ftr', columns=None, use_threads=True)
df2 = pd.read_feather('../Data/D3_EBAN_cleaned.ftr', columns=None, use_threads=True)

### Data inspection

In [ ]:
#df1.head(5)

In [ ]:
df2.sample(5)

In [ ]:
#print(df1.shape)
print(df2.shape)

# 2. Filtering and / deleting data unwanted for Clutsering

## Filtering Data for PM

In [ ]:
# no Filtering Needed
#df3 = df2.loc[(df2['HKONT'] == '300000')]
df3 = df2.copy()

### Del Data before clustering

We won't use some data that is used in PM for the clustering, hence it would drastically change 

an therefore create final dataset used

In [ ]:
# Del Timedata hence we focus the business context for clustering
df = df3.drop(['BANFN', 'ERDAT', 'BADAT', 'LFDAT', 'FRGDT', 'BEDAT','EBELP','EBELN'], axis=1).copy()

In [ ]:
cats_cat = list(df.select_dtypes(include=['category']).columns)
nums_cat = list(df.select_dtypes(include=['float64', 'int64']).columns)

## 2.1 Pre Clustering

In [ ]:
df_kp = df.copy()

In [ ]:
df_corc = df_kp.copy()

In [ ]:
cats_cat = df_corc[cats_cat]
        
for (columnName, columnData) in cats_cat.items():
    df_corc[columnName] = df_corc[columnName].astype(str)
    if isinstance(df_corc[columnName][0], str):
        s = pd.Series(df_corc[columnName], dtype="category")
        df_corc[columnName] = s.cat.rename_categories(np.arange(len(np.unique(df_corc[columnName]))))

In [ ]:
df_corc.head()

In [ ]:
df_corc.shape

## GAP Statistics

In [ ]:
# GAP
import fg_cleansing_helper_new as fg


#create a K-Proto Classifier
catColumnsPos = [df_corc.columns.get_loc(col) for col in list(df_corc.select_dtypes('category').columns)]
kprototype = KPrototypes(n_jobs = -1, init = 'Cao', random_state = 0, max_iter = 100, verbose=False)

#Compute gap with Classifier
k_max = 10
gap, reference_inertia, ondata_inertia = fg.compute_gap_k_proto(kprototype, df_corc.to_numpy(), catColumnsPos, k_max)

# plot results 1
plt.plot(range(1, k_max+1), reference_inertia,
         '-+', label='reference')
plt.plot(range(1, k_max+1), ondata_inertia,
         '-+', label='data')
plt.xlabel('k')
plt.ylabel('log(inertia)')
plt.show()

In [ ]:
# plot GAP Statistics
plt.plot(range(1, k_max+1), gap, '-o')
plt.ylabel('gap')
plt.xlabel('k')

## 3. K-Proto

In [ ]:
# now we know that there are apoximatly 2 maybe 3 clusters

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import pairwise_distances

from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler
random_state = 1234
pd.options.plotting.backend = "plotly"
#import timeit

# Convert dataframe to matrix
dfMatrix = df_corc.to_numpy()
# Choose optimal K using Elbow method
l_elbow = []
l_silh = []
l_chi1 = []
l_dbsc = []
print(dfMatrix.shape)
print(df_corc.shape)

#define cat vars for K-prototypes as numerical list
catColumnsPos = [df_corc.columns.get_loc(col) for col in list(df_corc.select_dtypes('category').columns)]
print(catColumnsPos)
print('Categorical columns: {}'.format(list(df_corc.select_dtypes('category').columns)))


for cluster in range(1, 15):
    try:
        kprototype = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Cao', random_state = 0, max_iter = 100, verbose=False) #Cao
        #print('step')
        kprototype.fit_predict(dfMatrix, categorical = catColumnsPos)
        print('cluster:' + str(cluster))
        l_elbow.append(kprototype.cost_) #inertia_
        

        print('Cluster initiation: {}'.format(cluster))
        #lets calc some scores
        if cluster > 1:
            l_chi1.append(metrics.calinski_harabasz_score(dfMatrix, kprototype.labels_))
            l_silh.append(metrics.silhouette_score(dfMatrix, kprototype.labels_, metric='euclidean'))
            l_dbsc.append(metrics.davies_bouldin_score(dfMatrix, kprototype.labels_))
        #test
        else:
            l_dbsc.append(0)
            l_chi1.append(0)
            l_silh.append(0)

    except:
        break
    

print('finished Kproto')

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

a = len(l_dbsc) + 1

df_chi = pd.DataFrame({'Cluster':range(1, a), 'CH value':l_chi1})
df_silh = pd.DataFrame({'Cluster':range(1, a), 'S Score':l_silh})
df_elbow = pd.DataFrame({'Cluster':range(1, a), 'Cost':l_elbow})
df_dbsc = pd.DataFrame({'Cluster':range(1, a), 'DB value':l_dbsc})

fig = make_subplots(rows=1, cols=4) #for titles above -->
subplot_titles=("CH value", "Silhouette index", "Elbow", 'DB value')

trace1 = px.line(df_chi, x="Cluster", y="CH value")
trace2 = px.line(df_silh, x="Cluster", y="S Score")
trace3 = px.line(df_elbow, x="Cluster", y="Cost")
trace4 = px.line(df_dbsc, x="Cluster", y="DB value")

#1
fig.add_trace(go.Scatter(mode='lines+markers', x=trace1.data[0]['x'], y=trace1.data[0]['y'],text=trace1.data[0]['x'], line = dict(color='#FEAEFE', width=3), marker=dict(size=8), legendgroup="Calinski Harabasz Index"), row=1, col=1, ) #, name="Chi"
#fig.update_traces(textposition='top right', line = dict(color='#FEAEFE', width=3), marker=dict(size=8))
fig['layout']['yaxis1']['title']='CH value'
fig['layout']['yaxis2']['title']='Silhouette Score'
fig['layout']['yaxis3']['title']='Elbow'
fig['layout']['yaxis4']['title']='DB Value'


#2
fig.add_trace(go.Scatter(mode='lines+markers', x=trace2.data[0]['x'], y=trace2.data[0]['y'], text=trace2.data[0]['x'], line = dict(color='#00F19D', width=3), marker=dict(size=8), legendgroup="Sillhouette Score"), row=1, col=2) #, name="Silhouette"
#fig.update_traces(textposition='top right')

#3
fig.add_trace(go.Scatter(mode='lines+markers', x=trace3.data[0]['x'], y=trace3.data[0]['y'], text=trace3.data[0]['x'], line = dict(color='#FC05FB', width=3), marker=dict(size=8), legendgroup="Elbow Method"), row=1, col=3) #, name="Elbow"
#fig.update_traces(textposition='top right')

#3
fig.add_trace(go.Scatter(mode='lines+markers', x=trace4.data[0]['x'], y=trace4.data[0]['y'], text=trace4.data[0]['x'], line = dict(color='#1F363D', width=3), marker=dict(size=8), legendgroup="DB value"), row=1, col=4) #, name="Elbow"
#fig.update_traces(textposition='top right')

# customization
fig.update_traces(textposition='top right')
#fig['layout']['xaxis1']['title']='CH value'
#fig['layout']['xaxis2']['title']='Silhouette index'
#fig['layout']['xaxis3']['title']='Elbow'

fig.update_xaxes(nticks=10, linecolor='#F5F5F5', gridcolor='#F5F5F5')
fig.update_yaxes(linecolor='#F5F5F5', gridcolor='#F5F5F5')
fig.update_layout(plot_bgcolor='white')
fig.update_layout(height=290, width=1200)
fig.update_layout(margin=dict(l=10, r=10, t=30, b=20))
#fig.update_layout(title_text="Clustering Metrics for K-Prototypes")
#fig.update_layout(showlegend=False)
fig.update_layout(legend=dict(
    orientation="h",
    xanchor="right",
    yanchor="bottom",
    y=-0.3, 
    x=1 
))
fig.update_layout(showlegend=False)
fig.show()

#export
#fig.write_image("EBAN_Metrics_placeholder.pdf")

# Final Clustering

## K-Proto

In [ ]:
# number of clusters
K = 2
# no. of points
N = df_corc.shape[0] 
# no. of dimensions
M = df_corc.shape[1]

# no. of numerical dimensions
#MN = 5
# no. of times test is repeated
#T = 3

kp = KPrototypes(n_clusters=K, init='Huang', n_init=1, verbose=False, n_jobs = -1)
kp.fit_predict(dfMatrix, categorical = catColumnsPos)

print(kp.cluster_centroids_)
print(kp.labels_)
print(kp.labels_.shape)

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import pairwise_distances
print(metrics.silhouette_score(dfMatrix, kp.labels_, metric='euclidean')) #> 0.5 should be good but higher is better
#print(metrics.silhouette_score(dfmatrix_f, kp.labels_, metric='precomputed')) #> 0.5 should be good
print(metrics.calinski_harabasz_score(dfMatrix, kp.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(dfMatrix, kp.labels_)) #lower ist better
print(np.unique(kp.labels_, return_counts=True))


In [ ]:
#add results to original Dataset 
df3['kproto_labels'] = kp.labels_

In [ ]:
#df3.sample(10)

### Cluster Metrics
- elbwo
- silloutte 
- gap statistics
- harabaz index


In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import pairwise_distances
print(metrics.silhouette_score(dfMatrix, kp.labels_, metric='euclidean')) #> 0.5 should be good but higher is better
#print(metrics.silhouette_score(dfmatrix_f, kp.labels_, metric='precomputed')) #> 0.5 should be good
print(metrics.calinski_harabasz_score(dfMatrix, kp.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(dfMatrix, kp.labels_)) #lower ist better
print(np.unique(kp.labels_, return_counts=True))


# 3. clustering with DBSCAN

In [ ]:
df_gw = df.iloc[:,:-1].copy()

In [ ]:
df.head(2)

In [ ]:
df_gw.head(3)

In [ ]:
import gower

distance_matrix = gower.gower_matrix(np.asarray(df_gw))

In [ ]:
from sklearn.cluster import DBSCAN

# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.3, 
                        min_samples=2, 
                        metric="precomputed")

# Fitting the clustering algorithm
dbscan_cluster.fit(distance_matrix)

# Adding the results to a new column in the dataframe
#df3["dbscan_gower"] = dbscan_cluster.labels_

In [ ]:
dbscan_cluster.labels_.shape

In [ ]:
#metrics
print(metrics.silhouette_score(distance_matrix, dbscan_cluster.labels_, metric='precomputed')) #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, dbscan_cluster.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, dbscan_cluster.labels_)) #lower ist better 
print(np.unique(dbscan_cluster.labels_, return_counts=True))

## DBSCAN with calculated Eps
- min samples should be arround No. of features
- Eps can be calculated

In [ ]:
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt

neighbors = NearestNeighbors(n_neighbors=30, metric='precomputed')
neighbors_fit = neighbors.fit(distance_matrix)
distances, indices = neighbors_fit.kneighbors(distance_matrix)

distances = np.sort(distances, axis=0)
distances = distances[:,1]
#plt.plot(distances)

In [ ]:
import plotly.express as px


#mypal2 =['#FEAEFE', '#FC05FB', '#B4FFE4', '#3FFEBA', '#1F363D', '#2B4570', '#3A2D32', '#706F6F']
fig = px.line(distances, 
              width=700, 
              height=500
              )
#linecolor
fig.update_traces(textposition='top right', line = dict(color='#FEAEFE', width=3), marker=dict(size=8))
#axes
fig.update_xaxes(nticks=10, linecolor='#F5F5F5', gridcolor='#F5F5F5')
fig.update_yaxes(linecolor='#F5F5F5', gridcolor='#F5F5F5')
#background
fig.update_layout(plot_bgcolor='white')
#show
fig.show()

In [ ]:
#Eps shoukd be on 0.008 BUT verry Steep Curve!

In [ ]:
from sklearn.cluster import DBSCAN

# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.05, #0.05
                        min_samples=6, #6 
                        metric="precomputed")

# Fitting the clustering algorithm
dbscan_cluster.fit(distance_matrix)

# Adding the results to a new column in the dataframe
#df3["dbscan_gower"] = dbscan_cluster.labels_

In [ ]:
print(metrics.silhouette_score(distance_matrix, dbscan_cluster.labels_, metric='precomputed')) #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, dbscan_cluster.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, dbscan_cluster.labels_)) #lower ist better 
print(np.unique(dbscan_cluster.labels_, return_counts=True))

In [ ]:
# Adding the last results to a new column in the dataframe
df3["dbscan_gower"] = dbscan_cluster.labels_

In [ ]:
# outlier vis
df3.loc[df3.dbscan_gower==-1]

##  4. Clustering with Aglomerative Clustering

In [ ]:
# Clustering algorithm
from sklearn.cluster import AgglomerativeClustering

# Rand Index
from sklearn.metrics.cluster import rand_score

# Encode labels
from sklearn import preprocessing

# Confusion Matrix
from sklearn.metrics import confusion_matrix

### Single Linkage

In [ ]:
#test 
model_single = AgglomerativeClustering(n_clusters=10, linkage='single', metric='precomputed')
clusters_single = model_single.fit_predict(distance_matrix)

In [ ]:
print(metrics.silhouette_score(distance_matrix, model_single.labels_, metric="precomputed")) #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, model_single.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, model_single.labels_)) #lower ist better 
print(np.unique(clusters_single, return_counts=True))

In [ ]:
# Adding the results to a new column in the dataframe
df3["AgloSngl_gower"] = clusters_single

### Average Linkage

In [ ]:
model_average = AgglomerativeClustering(n_clusters=10, linkage='average', metric='precomputed')
clusters_average = model_average.fit_predict(distance_matrix)

In [ ]:
print(metrics.silhouette_score(distance_matrix, model_average.labels_, metric="precomputed")) #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, model_average.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, model_average.labels_)) #lower ist better 
print(np.unique(clusters_average, return_counts=True))

In [ ]:
# Adding the results to a new column in the dataframe
df3["AgloAvg_gower"] = clusters_average

### Complete Linkage

In [ ]:
model_complete = AgglomerativeClustering(n_clusters=10, linkage='complete', metric='precomputed')
clusters_complete = model_complete.fit_predict(distance_matrix)

In [ ]:
print(metrics.silhouette_score(distance_matrix, model_complete.labels_, metric="precomputed"))  #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, model_complete.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, model_complete.labels_)) #lower ist better
print(np.unique(clusters_complete, return_counts=True))

In [ ]:
# Adding the results to a new column in the dataframe
df3["AgloComp_gower"] = clusters_complete

In [ ]:
#df3.loc[(df3.AgloComp_gower == 1) | (df3.AgloComp_gower == 6)]

In [ ]:
df3.sample(10)

## 5. Spectral clustering

In [ ]:
from sklearn.cluster import SpectralClustering


# EXPORT

In [ ]:
df3.sample(7)

In [ ]:
df3.to_feather('../Data/D3_EBAN_CL.ftr')